In [2]:
from utils import db
from utils import s3
import pandas as pd
import random
import datetime

In [11]:

""" 
1.Create table :tran_fact
# 2. Create static partition as dataset_date as varchar(10). Make sure tran_date = dataset_date-1
# PROBLEM: parquet requires column names and meta data


tran_id int, cust_id varchar(20),tran_date date,tran_ammount decimal(10,2), tran_type varchar(1)

tran_id : unique id of 6 character
cust_id : CA+ 4 integer
tran_date: is constant for one file
tran_ammount: number havin decimal(10,2)
tran_type :[C/D]
sate_cd :varchar(2)


102020,CA1001,2022-02-01,1200,C,CA
102021,CA1002,2022-02-01,700,C,NY
102022,CA1003,2022-02-01,500,C,NJ
102023,CA1004,2022-02-02,900,C,VA
102020,CA1001,2022-02-02,200,D,CA
&/

*/
"""

"""
create table if not exists cust_tran_fact
(
    tran_id varchar(10) , 
    cust_id varchar(20) ,
    tran_ammount decimal(10,2), 
    tran_type varchar(1) , 
    tran_stat_cd varchar(2) ,
    tran_date date
)
partitioned by (load_date varchar(10))
row format delimited fields terminated by ','
location "s3://quintrix-spearscjs/data/src_customer/cust_tran_fact/"
tblproperties ("skip.header.line.count"="1");

"""


# generate dummy data



AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [66]:

import pandas as pd
import random
from datetime import datetime, timedelta


start_date = datetime(2023, 1, 1)
number_of_days = 7
record_count_min = 1000
record_count_max = 10000

tran_types = ['C', 'D']
state_cds = ['CA', 'AZ', 'NJ', 'AL', 'AK', 'CO', 'KY', 'WV']


def generate_dummy_info(record_count: int, date: datetime, start_id = 0):
    print(f'Generating {record_count} records for {date.strftime("%Y-%m-%d")}')
    ran_data = []
    for i in range(1, record_count+1):
        tran_id = f'{start_id + i:06}'
        cust_id = f'CA{random.randint(0,9999):04}'
        tran_ammount = round(random.uniform(100.00, 99999999.99),2)
        tran_type = random.choice(tran_types)
        stat_cd = random.choice(state_cds)
        tran_date = date.strftime("%Y-%m-%d")
        ran_data.append((tran_id, cust_id, tran_ammount, tran_type, stat_cd, tran_date))

    df = pd.DataFrame(ran_data, columns=['tran_id', 'cust_id', 'tran_ammount', 'tran_type', 'tran_stat_cd', 'tran_date'])
    filename = f'data/src_customer/cust_tran_fact/daily/cust_tran_fact_{date.strftime("%Y-%m-%d")}.csv'
    # df.to_parquet(filename, engine="fastparquet", )
    df.to_csv(filename, index=False)

    return filename


def generate_for_days(min_record_num: int, max_record_num: int, start_date: datetime, days: int):
    cur_date = start_date
    filenames = []
    count = 0
    for i in range(days):
        num_records = random.randint(min_record_num, max_record_num)
        filename = generate_dummy_info(num_records, cur_date, count)
        print(filename)
        filenames.append((filename, cur_date.strftime("%Y-%m-%d")))
        cur_date += timedelta(days=1)
        count += num_records
    print(f'Data generated for {days} days')
    return filenames

filenames = generate_for_days(record_count_min, record_count_max, start_date, number_of_days)


Generating 8639 records for 2023-01-01
data/src_customer/cust_tran_fact/daily/cust_tran_fact_2023-01-01.csv
Generating 8209 records for 2023-01-02
data/src_customer/cust_tran_fact/daily/cust_tran_fact_2023-01-02.csv
Generating 2653 records for 2023-01-03
data/src_customer/cust_tran_fact/daily/cust_tran_fact_2023-01-03.csv
Generating 6999 records for 2023-01-04
data/src_customer/cust_tran_fact/daily/cust_tran_fact_2023-01-04.csv
Generating 6083 records for 2023-01-05
data/src_customer/cust_tran_fact/daily/cust_tran_fact_2023-01-05.csv
Generating 5448 records for 2023-01-06
data/src_customer/cust_tran_fact/daily/cust_tran_fact_2023-01-06.csv
Generating 6700 records for 2023-01-07
data/src_customer/cust_tran_fact/daily/cust_tran_fact_2023-01-07.csv
Data generated for 7 days


In [96]:
# upload data to s3
for f in filenames:
    filename = f[0]
    name = f[0].split('/')[4]
    # add upload date for s3 folder names / hive partitions (tran_date plus one)
    date = list(f[1])
    date[9] = str((int(date[9]) + 1 ) % 10)
    date = "".join(date)
    s3.upload_file(filename, 'quintrix-spearscjs', f'data/src_customer/cust_tran_fact/load_date={date}/{name}')

In [ ]:
# 3. Add 7 partitions dataset_date

""" 
set hivevar:src_schema=src_customer;
use ${hivevar:src_schema};
alter table cust_tran_fact add partition (load_date='2023-01-02');
alter table cust_tran_fact add partition (load_date='2023-01-03');
alter table cust_tran_fact add partition (load_date='2023-01-04');
alter table cust_tran_fact add partition (load_date='2023-01-05');
alter table cust_tran_fact add partition (load_date='2023-01-06');
alter table cust_tran_fact add partition (load_date='2023-01-07');
alter table cust_tran_fact add partition (load_date='2023-01-08');
"""

In [ ]:


""" 
/*
Dataquality check:
1. create a table as table_states:
cols:
database_name varchar(20),
table_name varchar(50),
partition_key varchar(30),
rec_count int(10),
load_date date,
execution_key varchar(100)  >>> Partition col

execution_key is (database_name + "-"+table_name+"-"+partition_key


2. After each load into tran_fact run the query to load data into table_states
query is : get the count from tran_fact an load into table_states

*/
""" 
""" 

create table if not exists table_states (
    database_name varchar(20),
    table_name varchar(50),
    partition_key varchar(30),
    rec_count int,
    load_date date
)
partitioned by (execution_key varchar(100))
stored as parquet
location "s3://quintrix-spearscjs/data/src_customer/cust_tran_fact/";





insert overwrite table_states( 
 database_name varchar(20),
 table_name varchar(50),
 partition_key varchar(30),
 rec_count int,
 load_date date
)
select 'src_customer' database_name, 
 'cust_tran_fact' table_name, 
 'load_date' partition_key, 
 count(1) rec_count,
 load_date load_date

from cust_tran_fact 
GROUP BY load_date;


"""


""" 

-- 1. Total unique customer per day.
SELECT tran_date, COUNT(DISTINCT cust_id) unique_customers 
FROM cust_tran_fact 
GROUP BY tran_date;

-- 2. Total number of unique customer till date (BROKEN FOR EMR)
/*
rank -- partition by cust order date


*/
-- using sum
WITH ranks AS (
    SELECT DISTINCT cust_id, tran_date, RANK() OVER(PARTITION BY cust_id ORDER BY tran_date)
    FROM cust_tran_fact
)
SELECT DISTINCT tran_date, 
    SUM(CASE WHEN rank = 1 THEN 1 ELSE 0 END) OVER(ORDER BY tran_date) total_unique_cust 
FROM ranks
 ORDER BY tran_date;


-- using nested select
SELECT DISTINCT tran_date,
       (SELECT COUNT(DISTINCT cust_id) FROM cust_tran_fact  WHERE tran_date <= t.tran_date) total_unique_cust
FROM cust_tran_fact t
-- GROUP BY tran_date
ORDER BY tran_date;



-- 3. Total transaction amount per customer per day ( if its C then add if D then subtract )
SELECT tran_date, cust_id, SUM(tran_ammt) total_tran_ammt
FROM
    (SELECT tran_date, cust_id, 
        CASE 
            WHEN tran_type = 'C'
                THEN tran_ammount
            ELSE -tran_ammount
        END AS tran_ammt
    FROM cust_tran_fact) t
GROUP BY tran_date, cust_id



-- 4. Find out duplicate transaction in total.
SELECT tran_id, cust_id,  tran_date, tran_ammount, tran_type
FROM cust_tran_fact tf
GROUP BY tran_id, cust_id,  tran_date, tran_ammount, tran_type
HAVING COUNT(*) > 1;



-- 5. show the transaction which has debit but never credit before. (BROKEN FOR HIVE EMR) 
WITH counts AS(
    SELECT tran_id, tran_type, tran_date,
    -- count all 'C' type transactions with same id that happened before tran_date (if it is 0, there were no credits before the transaction) 
    (
        SELECT COUNT(tran_id) FROM cust_tran_fact 
        WHERE tran_date <= tf.tran_date AND tran_id = tf.tran_id AND tran_type = 'C'
    ) FROM cust_tran_fact tf)
SELECT tran_id, tran_date FROM counts WHERE tran_type = 'D' AND count = 0;
"""
